In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.preprocessing.image import load_img
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, Dense, MaxPooling2D, Flatten
import random
import os
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from PIL import Image

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import zipfile

with zipfile.ZipFile("../input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")
    
with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip","r") as z:
    z.extractall(".")

In [ ]:
train_dir = "./train/"
test_dir = "./test1/"

Dtrain = os.listdir(train_dir)
categories = []
for filename in Dtrain:
    category = filename.split(".")[0]
    if category == "dog":
        categories.append("dog")
    else:
        categories.append("cat")
df = pd.DataFrame({
    "filename" : Dtrain,
    "category" : categories
})

In [ ]:
print(df.head())
print(df.tail())

In [ ]:
dfCat = df[df["category"] == 'cat'].head(2000)
dfTestCat = dfCat.head(500)
dfCat = dfCat.iloc[500:,]
dfValCat = dfCat.head(500)
dfCat = dfCat.iloc[500:,]
print(dfCat.shape)
print(dfValCat.shape)

In [ ]:
dfDog = df[df["category"] == 'dog'].head(2000)
dfTestDog = dfDog.head(500)
dfDog = dfDog.iloc[500:,]
dfValDog = dfDog.head(500)
dfDog = dfDog.iloc[500:,]

In [ ]:
frames = [dfCat, dfDog] 
frames2 = [dfValCat, dfValDog]
frames3 = [dfTestCat, dfTestDog]

In [ ]:
df2 = pd.concat(frames)
df2Val = pd.concat(frames2)
df2Test= pd.concat(frames3)
print(df2.head())
print(df2.tail())

In [ ]:
# I need shuffle this data
#The frac keyword argument specifies the fraction of rows to return in the random sample, so frac=1 means return all rows (in random order).
#Here, specifying drop=True prevents .reset_index from creating a column containing the old index entries.
df2 = df2.sample(frac=1).reset_index(drop=True)
df2Val = df2Val.sample(frac=1).reset_index(drop=True)
df2Test = df2Test.sample(frac=1).reset_index(drop=True)

In [ ]:
df2

In [ ]:
print(df2.head())
print(df2.tail())

In [ ]:
#I should it named test not validate because during training model I split 0,25 of data as validation data
#from sklearn.model_selection import train_test_split

#train_df, validate_df = train_test_split(df2, test_size = 0.25, random_state = 42)
#train_df = train_df.reset_index(drop=True)
#validate_df = validate_df.reset_index(drop=True)

In [ ]:
df2.category.value_counts().plot.bar()
plt.title(str(len(df2[df2['category'] == 'cat']))+" cats" + " | " +str(len(df2[df2['category'] == 'dog']))+" dogs", fontsize=18)
plt.ylabel(len(df2), fontsize = 14)
plt.show()

In [ ]:
df2Val.category.value_counts().plot.bar()
plt.ylabel(len(df2Val), fontsize = 14)
plt.title(str(len(df2Val[df2Val['category'] == 'dog']))+" Dogs  |  "+str((len(df2Val[df2Val['category'] == 'cat'])))+" Cats", fontsize=18)
plt.show()

In [ ]:
df2Test.category.value_counts().plot.bar()
plt.ylabel(len(df2Test), fontsize = 14)
plt.title(str(len(df2Test[df2Test['category'] == 'cat']))+" Cats  |  "+str((len(df2Test[df2Test['category'] == 'dog'])))+" Dogs", fontsize=18)
plt.show()

In [ ]:
import pandas as pd
main_dir = "/kaggle/working/"
train_dir = "train"
path = os.path.join(main_dir,train_dir)
convert = lambda category : int(category == 'dog')
X=[]
y=[]

 
def create_test_data(path):
    for t in df2['filename']:
        print(t)
        category = t.split(".")[0]
        category = convert(category)
        img_array = cv2.imread(os.path.join(path,t),cv2.IMREAD_GRAYSCALE)
        print(img_array)
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        X.append(new_img_array)
        y.append(category)

In [ ]:
create_test_data(path)
X_train = np.array(X).reshape(-1, 80,80,1)
y_train = np.array(y)

In [ ]:
X_test=[]
y_test=[]
#def create_test_data(path):
    for t in df2Test['filename']:
        print(t)
        category = t.split(".")[0]
        category = convert(category)
        img_array = cv2.imread(os.path.join(path,t),cv2.IMREAD_GRAYSCALE)
        print(img_array)
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        X_test.append(new_img_array)
        y_test.append(category)

In [ ]:
create_test_data(path)
X_test = np.array(X_test).reshape(-1, 80,80,1)
y_test = np.array(y_test)

In [ ]:
X_val=[]
y_val=[]
#def create_test_data(path):
    for t in df2Val['filename']:
        print(t)
        category = t.split(".")[0]
        category = convert(category)
        img_array = cv2.imread(os.path.join(path,t),cv2.IMREAD_GRAYSCALE)
        print(img_array)
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        X_val.append(new_img_array)
        y_val.append(category)

In [ ]:
create_test_data(path)
X_val = np.array(X_val).reshape(-1, 80,80,1)
y_val = np.array(y_val)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_train[1])
print(y_train[1])
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
#Normalize data - as I can see abowe, every pixel is in scale 0-255 it's not optimized to process by comp, better is 0-1
X_train = X_train/255.0
X_test = X_test/255.0
X_val = X_val/255.0

In [ ]:
print(X_train[1])

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3), activation = 'relu', input_shape = X_train.shape[1:]))
model.add((MaxPooling2D(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation="relu"))
model.add((MaxPooling2D(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation="relu"))
model.add((MaxPooling2D(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(32, activation="relu"))

model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
import tensorflow as tf

dot_img_file = '/tmp/model_1.png'
tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

In [ ]:
import keras
from keras.utils import np_utils

num_classes=len(np.unique(y_train))
y_train=keras.utils.to_categorical(y_train, num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)
y_val=keras.utils.to_categorical(y_val,num_classes)

In [ ]:
model = Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(Conv2D(64,(3,3), activation = 'relu', input_shape = X_train.shape[1:]))
model.add((MaxPooling2D(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation="relu"))
model.add((MaxPooling2D(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), activation="relu"))
model.add((MaxPooling2D(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.25))

model.add(Dense(2, activation="sigmoid"))

model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history= model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val,y_val))

In [ ]:
accuracy = model.evaluate(X_test, y_test, verbose = 1)
print('\n','Test_Accuracy:-', accuracy[1])

In [ ]:
hist = pd.DataFrame(history.history)
print(hist)
hist['epoch'] = history.epoch

In [ ]:
import matplotlib.pyplot as plt

def plot_history():
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Catecorical crossentropy')
    plt.plot(hist['epoch'], hist['val_accuracy'], label='Val Accuracy')
    plt.plot(hist['epoch'], hist['val_loss'], label = 'Val Loss')
    plt.legend()
    plt.ylim([0,2])

plot_history()